In [2]:
import aiohttp
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import json
import asyncio

In [3]:
async def parse_website_meta(url: str) -> dict:
    """
    웹사이트에서 favicon과 site_name을 파싱하는 함수
    
    Args:
        url: 파싱할 웹사이트 URL
        
    Returns:
        dict: {
            "favicon": "favicon URL 또는 빈 문자열",
            "site_name": "사이트명 또는 빈 문자열"
        }
    """
    result = {"favicon": "", "site_name": ""}
    
    try:
        # HTTP 요청 타임아웃 설정
        timeout = aiohttp.ClientTimeout(total=5)
        
        async with aiohttp.ClientSession(timeout=timeout) as session:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            
            async with session.get(url, headers=headers) as response:
                if response.status != 200:
                    print(f"웹사이트 접근 실패: {url}, 상태코드: {response.status}")
                    return result
                
                html_content = await response.text()
                print(f"html_content : {html_content}")
                soup = BeautifulSoup(html_content, 'html.parser')
                print(f"soup : {soup}")
                
                # favicon 파싱
                favicon_url = await _parse_favicon(soup, url)
                print(f"favicon_url : {favicon_url}")
                if favicon_url:
                    result["favicon"] = favicon_url
                
                # site_name 파싱
                site_name = await _parse_site_name(soup)
                print(f"site_name : {site_name}")
                if site_name:
                    result["site_name"] = site_name
                else:
                    # site_name이 없으면 title 사용
                    title = await _parse_title(soup)
                    print(f"title : {title}")
                    if title:
                        result["site_name"] = title
    
    except asyncio.TimeoutError:
        print(f"웹사이트 메타 파싱 타임아웃: {url}")
    except Exception as e:
        print(f"웹사이트 메타 파싱 중 오류: {url}, 오류: {str(e)}")
    
    return result


async def _parse_favicon(soup: BeautifulSoup, base_url: str) -> str:
    """
    HTML에서 favicon을 찾는 함수 (더 포괄적인 선택자 지원)
    """
    try:
        # 1단계: link 태그에서 favicon 찾기 (우선순위 순)
        favicon_selectors = [
            # 일반적인 favicon
            'link[rel="icon"]',
            'link[rel="shortcut icon"]',
            # 다양한 크기의 PNG 아이콘
            'link[rel="icon"][sizes]',
            'link[rel="icon"][type="image/png"]',
            'link[rel="icon"][type="image/svg+xml"]',
            'link[rel="icon"][type="image/x-icon"]',
            # Apple 관련
            'link[rel="apple-touch-icon"]',
            'link[rel="apple-touch-icon-precomposed"]',
            # 특수한 형태들
            'link[rel="mask-icon"]',
            'link[rel="fluid-icon"]',
            # Microsoft 관련
            'link[rel="msapplication-TileImage"]'
        ]
        
        # 가장 적합한 favicon 찾기
        best_favicon = None
        best_priority = -1
        
        all_favicons = []
        for i, selector in enumerate(favicon_selectors):
            favicon_links = soup.select(selector)
            print(f"favicon_links : {favicon_links}")
            for favicon_link in favicon_links:
                href = favicon_link.get('href')
                if href:
                    all_favicons.append({
                        'href': href,
                        'priority': i,
                        'sizes': favicon_link.get('sizes'),
                        'type': favicon_link.get('type'),
                        'rel': favicon_link.get('rel')
                    })
        
        # 우선순위와 크기를 고려하여 최적의 favicon 선택
        if all_favicons:
            # 32x32 또는 16x16 크기를 우선으로 선택
            preferred_sizes = ['32x32', '16x16', '64x64', '48x48']
            
            for size in preferred_sizes:
                for favicon in all_favicons:
                    if favicon['sizes'] == size:
                        return urljoin(base_url, favicon['href'])
            
            # 크기 정보가 없으면 우선순위가 높은 것 선택
            all_favicons.sort(key=lambda x: x['priority'])
            print(f"all_favicons : {all_favicons}")
            return urljoin(base_url, all_favicons[0]['href'])
        # 2단계: meta 태그에서 Microsoft 타일 이미지 찾기
        meta_favicon_selectors = [
            'meta[name="msapplication-TileImage"]',
            'meta[property="og:image"]'  # 최후의 수단
        ]
        
        for selector in meta_favicon_selectors:
            meta_tag = soup.select_one(selector)
            if meta_tag and meta_tag.get('content'):
                content = meta_tag.get('content')
                if content.lower().endswith(('.ico', '.png', '.svg', '.jpg', '.jpeg', '.gif')):
                    return urljoin(base_url, content)
        
        # 3단계: 기본 favicon 경로들 시도
        parsed_url = urlparse(base_url)
        default_favicon_paths = [
            '/favicon.ico',
            '/favicon.png',
            '/favicon.svg',
            '/favicon-32x32.png',
            '/favicon-16x16.png',
            '/favicon_icon.png',
            '/assets/favicon.ico',
            '/assets/images/favicon.ico',
            '/images/favicon.ico',
            '/static/favicon.ico',
            '/web/upload/favicon.ico',  # 사용자가 제공한 예시 경로 패턴
            '/public/favicon.ico'
        ]
        
        # 첫 번째 기본 경로 반환 (실제 존재 여부는 클라이언트에서 확인)
        if default_favicon_paths:
            return f"{parsed_url.scheme}://{parsed_url.netloc}{default_favicon_paths[0]}"
        
    except Exception as e:
        print(f"favicon 파싱 중 오류: {str(e)}")
    
    return ""


async def _parse_site_name(soup: BeautifulSoup) -> str:
    """
    HTML에서 site_name을 찾는 함수 (더 포괄적인 소스 지원)
    """
    try:
        # 1단계: 가장 정확한 site_name 소스들 (우선순위 단계별)
        primary_selectors = [
            'meta[property="og:site_name"]',
            'meta[name="application-name"]',
            'meta[name="apple-mobile-web-app-title"]'
        ]
        
        for selector in primary_selectors:
            meta_tag = soup.select_one(selector)
            if meta_tag:
                content = meta_tag.get('content')
                if content and content.strip():
                    return content.strip()
        
        # 2단계: 소셜 미디어 관련 site_name
        social_selectors = [
            'meta[property="twitter:site"]',
            'meta[name="twitter:site"]',
            'meta[property="twitter:creator"]',
            'meta[name="twitter:creator"]'
        ]
        
        for selector in social_selectors:
            meta_tag = soup.select_one(selector)
            if meta_tag:
                content = meta_tag.get('content')
                if content and content.strip():
                    # Twitter handle에서 @를 제거
                    if content.startswith('@'):
                        content = content[1:]
                    return content.strip()
        
        # 3단계: 기타 메타 정보에서 site_name 추출
        other_selectors = [
            'meta[name="generator"]',  # WordPress, Wix 등
            'meta[name="author"]',
            'meta[name="publisher"]',
            'meta[property="article:publisher"]',
            'meta[name="copyright"]',
            'meta[name="DC.publisher"]',  # Dublin Core
            'meta[name="DC.creator"]'
        ]
        
        for selector in other_selectors:
            meta_tag = soup.select_one(selector)
            if meta_tag:
                content = meta_tag.get('content')
                if content and content.strip():
                    # 일부 generator 태그는 버전 정보를 포함하므로 정리
                    if 'generator' in selector:
                        # "WordPress 6.3" -> "WordPress" 같은 경우
                        content = content.split()[0] if content.split() else content
                    return content.strip()
        
        # 4단계: JSON-LD 구조화 데이터에서 site_name 찾기
        json_ld_scripts = soup.find_all('script', type='application/ld+json')
        for script in json_ld_scripts:
            try:
                if script.string:
                    data = json.loads(script.string)
                    # 단일 객체 또는 배열 처리
                    if isinstance(data, list):
                        data = data[0] if data else {}
                    
                    # 다양한 JSON-LD 프로퍼티에서 site_name 찾기
                    possible_names = [
                        data.get('publisher', {}).get('name') if isinstance(data.get('publisher'), dict) else data.get('publisher'),
                        data.get('author', {}).get('name') if isinstance(data.get('author'), dict) else data.get('author'),
                        data.get('name'),
                        data.get('alternateName'),
                        data.get('brand', {}).get('name') if isinstance(data.get('brand'), dict) else data.get('brand')
                    ]
                    
                    for name in possible_names:
                        if name and isinstance(name, str) and name.strip():
                            return name.strip()
            except (json.JSONDecodeError, AttributeError):
                continue
        
        # 5단계: 페이지 요소에서 site_name 추출
        element_selectors = [
            'h1.site-title',
            'h1.logo',
            '.site-name',
            '.site-title',
            '.logo-text',
            'header h1',
            'nav .brand',
            '.navbar-brand',
            'header .brand'
        ]
        
        for selector in element_selectors:
            element = soup.select_one(selector)
            if element:
                text = element.get_text(strip=True)
                if text and len(text) < 100:  # 너무 긴 텍스트는 제외
                    return text

    except Exception as e:
        print(f"site_name 파싱 중 오류: {str(e)}")
    
    return ""


async def _parse_title(soup: BeautifulSoup) -> str:
    """
    HTML에서 title을 찾는 함수
    """
    try:
        # og:title 우선, 없으면 일반 title
        og_title = soup.select_one('meta[property="og:title"]')
        if og_title and og_title.get('content'):
            return og_title.get('content').strip()
        
        title_tag = soup.select_one('title')
        if title_tag and title_tag.get_text():
            return title_tag.get_text().strip()
    
    except Exception as e:
        print(f"title 파싱 중 오류: {str(e)}")
    
    return ""

async def _validate_favicon_url(session: aiohttp.ClientSession, favicon_url: str) -> bool:
    """Favicon URL이 실제로 존재하는지 확인"""
    try:
        async with session.head(favicon_url, timeout=aiohttp.ClientTimeout(total=3)) as response:
            return response.status == 200 and 'image' in response.headers.get('content-type', '')
    except:
        return False

In [21]:
# url = "https://nara1.kr/"
# url = "https://futuresnow.gitbook.io/newstoday/2025-05-14/news/today/bloomberg"
# url = "https://dream.kotra.or.kr/kotranews/cms/news/actionKotraBoardDetail.do?SITE_NO=3&MENU_ID=110&CONTENTS_NO=1&bbsGbn=245&bbsSn=245&pNttSn=230076"
# url = "https://korean.cri.cn/video/news"
url = "https://chatty.kr/"

In [22]:

# 웹사이트 메타 정보 파싱 (favicon, site_name)
meta_info = await parse_website_meta(url)
favicon = meta_info.get("favicon", "")
site_name = meta_info.get("site_name", "")

print(f"meta_info : {meta_info}")
print(f"favicon : {favicon}")
print(f"site_name : {site_name}")

html_content : <!doctype html>
<html lang="ko">
<head>  
	<title>AI 프로 _ chatty.kr</title>  
	<link rel="preload" href="https://fastly.jsdelivr.net/gh/Project-Noonnu/noonfonts_2107@1.1/Pretendard-Regular.woff" as="font" type="font/woff" crossorigin="anonymous">
	<meta charset="UTF-8">
	<link rel="stylesheet" type="text/css" href="/css/animate.min.css">
	<link rel="stylesheet" type="text/css" href="/css/swiper-bundle.min.css">
	<link rel="stylesheet" type="text/css" href="/css/fonts.css">
	<link rel="stylesheet" type="text/css" href="/css/layout2.css">
	<link rel="stylesheet" type="text/css" href="/css/layout.css">
	<meta http-equiv="X-UA-Compatible" content="IE=Edge">
	<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0">
	<meta name="keywords" content="맞춤형AI챗봇, Home, 학습형챗봇, 인공지능챗봇, 채팅 서비스, AI 챗봇 추천, 고객 맞춤형 AI, 채티, chatty">  
	<meta name="description" content="채티는 텍스트, 파일, 웹을 학습하여 나만을 위한 맞춤형 AI 대화를 제공합니다. 맞춤형 AI 챗봇 서비스로 고객 맞춤 대화를 경

In [23]:
async def create_search_snippets(soup: BeautifulSoup, search_query: str, max_snippets: int = 3, snippet_length: int = 150) -> list[dict]:
    """
    웹사이트에서 검색어와 관련된 텍스트 스니펫을 추출하는 함수
    
    Args:
        soup: BeautifulSoup 객체
        search_query: 검색어
        max_snippets: 반환할 최대 스니펫 수
        snippet_length: 각 스니펫의 최대 길이
        
    Returns:
        list[dict]: [
            {
                "text": "스니펫 텍스트",
                "source": "텍스트 출처 (예: title, description, heading 등)",
                "relevance_score": 점수
            },
            ...
        ]
    """
    snippets = []
    search_terms = search_query.lower().split()
    
    try:
        # 1단계: 메타 데이터에서 관련 텍스트 추출
        meta_selectors = {
            'title': 'title',
            'description': 'meta[name="description"]',
            'keywords': 'meta[name="keywords"]',
            'og:title': 'meta[property="og:title"]',
            'og:description': 'meta[property="og:description"]'
        }
        
        for source, selector in meta_selectors.items():
            elements = soup.select(selector)
            for element in elements:
                text = element.get_text().strip() if source == 'title' else element.get('content', '').strip()
                if text:
                    score = _calculate_relevance_score(text, search_terms)
                    if score > 0:
                        snippet = _create_snippet(text, search_terms, snippet_length)
                        snippets.append({
                            "text": snippet,
                            "source": source,
                            "relevance_score": score
                        })
        
        # 2단계: 주요 텍스트 컨텐츠에서 추출
        content_selectors = {
            'heading': ['h1', 'h2', 'h3'],
            'paragraph': ['p'],
            'article': ['article'],
            'section': ['section'],
            'list': ['li']
        }
        
        for source, selectors in content_selectors.items():
            for selector in selectors:
                elements = soup.select(selector)
                for element in elements:
                    text = element.get_text().strip()
                    if text and len(text) >= 10:  # 너무 짧은 텍스트 제외
                        score = _calculate_relevance_score(text, search_terms)
                        if score > 0:
                            snippet = _create_snippet(text, search_terms, snippet_length)
                            snippets.append({
                                "text": snippet,
                                "source": f"{source}",
                                "relevance_score": score
                            })
        
        # 3단계: JSON-LD 데이터에서 추출
        json_ld_scripts = soup.find_all('script', type='application/ld+json')
        for script in json_ld_scripts:
            try:
                if script.string:
                    data = json.loads(script.string)
                    if isinstance(data, list):
                        data = data[0] if data else {}
                    
                    text_fields = [
                        data.get('description', ''),
                        data.get('articleBody', ''),
                        data.get('text', '')
                    ]
                    
                    for text in text_fields:
                        if isinstance(text, str) and text.strip():
                            score = _calculate_relevance_score(text, search_terms)
                            if score > 0:
                                snippet = _create_snippet(text, search_terms, snippet_length)
                                snippets.append({
                                    "text": snippet,
                                    "source": "structured_data",
                                    "relevance_score": score
                                })
            except (json.JSONDecodeError, AttributeError):
                continue
        
        # 결과 정렬 및 필터링
        snippets.sort(key=lambda x: x['relevance_score'], reverse=True)
        return snippets[:max_snippets]
    
    except Exception as e:
        print(f"스니펫 생성 중 오류 발생: {str(e)}")
        return []

def _calculate_relevance_score(text: str, search_terms: list[str]) -> float:
    """
    텍스트와 검색어의 관련성 점수를 계산
    
    Args:
        text: 대상 텍스트
        search_terms: 검색어 리스트
        
    Returns:
        float: 관련성 점수 (0.0 ~ 1.0)
    """
    text_lower = text.lower()
    score = 0.0
    
    # 1. 정확한 구문 매칭
    exact_phrase = ' '.join(search_terms)
    if exact_phrase in text_lower:
        score += 1.0
    
    # 2. 개별 검색어 매칭
    matched_terms = sum(1 for term in search_terms if term in text_lower)
    score += (matched_terms / len(search_terms)) * 0.5
    
    # 3. 검색어 근접성 보너스
    words = text_lower.split()
    term_positions = []
    for term in search_terms:
        positions = [i for i, word in enumerate(words) if term in word]
        if positions:
            term_positions.extend(positions)
    
    if term_positions:
        term_positions.sort()
        if len(term_positions) > 1:
            max_gap = term_positions[-1] - term_positions[0]
            proximity_score = 1.0 / (max_gap + 1)
            score += proximity_score * 0.3
    
    # 4. 텍스트 위치 가중치
    if len(words) > 0:
        first_match = min(term_positions) if term_positions else len(words)
        position_weight = 1.0 - (first_match / len(words))
        score += position_weight * 0.2
    
    return min(1.0, score)

def _create_snippet(text: str, search_terms: list[str], max_length: int) -> str:
    """
    검색어를 포함하는 문맥 있는 스니펫 생성
    
    Args:
        text: 원본 텍스트
        search_terms: 검색어 리스트
        max_length: 최대 스니펫 길이
        
    Returns:
        str: 생성된 스니펫
    """
    text_lower = text.lower()
    
    # 검색어와 가장 관련있는 부분 찾기
    best_start = 0
    best_score = -1
    
    words = text.split()
    for i in range(len(words)):
        window = ' '.join(words[i:i + max_length // 10])  # 단어 기준으로 윈도우 생성
        score = sum(term in window.lower() for term in search_terms)
        if score > best_score:
            best_score = score
            best_start = i
    
    # 스니펫 추출
    start_pos = max(0, best_start)
    end_pos = min(len(words), start_pos + max_length // 5)
    snippet = ' '.join(words[start_pos:end_pos])
    
    # 스니펫이 문장 중간에서 시작하거나 끝나는 경우 처리
    if start_pos > 0:
        snippet = f"...{snippet}"
    if end_pos < len(words):
        snippet = f"{snippet}..."
    
    return snippet

In [26]:
async def search_naver_content(search_query: str, max_snippets: int = 3) -> None:
    """
    네이버 웹사이트에서 검색어와 관련된 텍스트를 추출하고 검색하는 함수
    
    Args:
        search_query: 검색어
        max_snippets: 반환할 최대 스니펫 수
    """
    url = "https://www.naver.com"
    
    try:
        # 커스텀 헤더 설정 (네이버는 User-Agent 확인)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        }
        
        async with aiohttp.ClientSession() as session:
            async with session.get(url, headers=headers) as response:
                if response.status != 200:
                    print(f"네이버 접근 실패: 상태 코드 {response.status}")
                    return
                
                html_content = await response.text()
                soup = BeautifulSoup(html_content, 'html.parser')
                
                # 검색 스니펫 생성
                snippets = await create_search_snippets(
                    soup=soup,
                    search_query=search_query,
                    max_snippets=max_snippets,
                    snippet_length=200  # 네이버는 좀 더 긴 스니펫이 유용할 수 있음
                )
                
                # 결과 출력
                if snippets:
                    print(f"\n[네이버 검색 결과: '{search_query}']\n")
                    for i, snippet in enumerate(snippets, 1):
                        print(f"[스니펫 {i}]")
                        print(f"텍스트: {snippet['text']}")
                        print(f"출처: {snippet['source']}")
                        print(f"관련성 점수: {snippet['relevance_score']:.2f}")
                        print("-" * 50)
                else:
                    print(f"\n검색어 '{search_query}'에 대한 결과를 찾을 수 없습니다.")

    except aiohttp.ClientError as e:
        print(f"네트워크 오류: {str(e)}")
    except Exception as e:
        print(f"검색 중 오류 발생: {str(e)}")

    

In [27]:
# 검색어 예시
search_queries = [
    "뉴스",
    "쇼핑",
    "메일"
]

# 각 검색어에 대해 검색 실행
for query in search_queries:
    await search_naver_content(query)
    print("\n" + "=" * 70 + "\n")


검색어 '뉴스'에 대한 결과를 찾을 수 없습니다.



검색어 '쇼핑'에 대한 결과를 찾을 수 없습니다.



검색어 '메일'에 대한 결과를 찾을 수 없습니다.




In [8]:
url2 = "https://www.chatty.kr"
# 커스텀 헤더 설정 (네이버는 User-Agent 확인)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

async with aiohttp.ClientSession() as session:
    async with session.get(url2, headers=headers) as response:
        if response.status != 200:
            print(f"네이버 접근 실패: 상태 코드 {response.status}")
        
        html_content = await response.text()
        soup = BeautifulSoup(html_content, 'html.parser')

In [32]:
snippets = []
search_terms = url2.lower().split()
# 1단계: 메타 데이터에서 관련 텍스트 추출
meta_selectors = {
    'title': 'title',
    'description': 'meta[name="description"]',
    'keywords': 'meta[name="keywords"]',
    'og:title': 'meta[property="og:title"]',
    'og:description': 'meta[property="og:description"]'
}
for source, selector in meta_selectors.items():
    elements = soup.select(selector)
    for element in elements:
        text = element.get_text().strip() if source == 'title' else element.get('content', '').strip()
        if text:
            score = _calculate_relevance_score(text, search_terms)
            if score > 0:
                snippet = _create_snippet(text, search_terms)
                snippets.append({
                    "text": snippet,
                    "source": source,
                    "relevance_score": score
                })

In [33]:
for source, selector in meta_selectors.items():
    elements = soup.select(selector)
    for element in elements:
        text = element.get_text().strip() if source == 'title' else element.get('content', '').strip()
        print(text)

AI 프로 chatty.kr
채티는 텍스트, 파일, 웹을 학습하여 나만을 위한 맞춤형 AI 대화를 제공합니다. 맞춤형 AI 챗봇 서비스로 고객 맞춤 대화를 경험해 보세요.
맞춤형AI챗봇, Home, 학습형챗봇, 인공지능챗봇, 채팅 서비스, AI 챗봇 추천, 고객 맞춤형 AI, 채티, chatty
맞춤형 AI 챗봇 채티
채티는 텍스트, 파일, 웹을 학습하여 나만을 위한 맞춤형 AI 대화를 제공합니다. 맞춤형 AI 챗봇 서비스로 고객 맞춤 대화를 경험해 보세요.


In [41]:
# 2단계: 주요 텍스트 컨텐츠에서 추출
content_selectors = {
    'heading': ['h1', 'h2', 'h3'],
    'paragraph': ['p'],
    'article': ['article'],
    'section': ['section'],
    'list': ['li']
}
texts = 0
for source, selectors in content_selectors.items():
    for selector in selectors:
        elements = soup.select(selector)
        for element in elements:
            text = element.get_text().strip()
            if text and len(text) >= 10 and texts <= 1000:  # 너무 짧은 텍스트 제외
                print(text)
                print(len(text))
                texts += len(text)

print(texts)

AI프로는 업무용 AI 솔루션입니다.
20
약 100개의 기업과 기관에서 사용하고 있습니다.
27
당신의 업무에 AI 날개를 달아드립니다.우리 기업/기관을 위한 맞춤형 AI 솔루션, AI프로
51
맞춤형 AI 챗봇 사용 신청서
16
기업이 필요한 모든 AI 기능을 한곳에 모아채팅, 검색엔진, 자료실, 번역, 문서요약, 이미지 등챗GPT, 클로드, 제미나이, 딥시크 등 마음대로 선택RAG 검색, LLM 검색, Web 검색 모두 가능
112
(우편번호: 08507) 서울특별시 금천구 가산디지털1로 168 우림라이온스밸리 A동 806호
52
Copyrightⓒ chatty.kr All rights reserved 개인정보처리방침
49
이 신청서는 맞춤형 AI 챗봇 사용을 희망하시는 분들을 위해 필요한 정보를 확인하고자 마련된 양식입니다.지금 신청하시면 1년간 '무료'로 서비스를 제공합니다.신청서를 작성해 주시면 확인 후 연락드리겠습니다.
115
[개인정보의 수집 및 이용 동의] 사용자는 맞춤형 AI 챗봇 서비스 제공 회사가 사용자 성명, 휴대폰, 이메일, 회사명 등 개인정보를 수집하고 이용하는 것에 동의 합니다. 내용보기
100
나보다 나를 더 잘 아는 RAG 기반의 똑똑한 AI 비서시나리오 기반의 기존 챗봇과 달리 답변 내용이 매우 우수합니다
65
우리 기업∙기관의 데이터를 학습시킬 수 있습니다.(예: “우리 회사 김한국 과장에 대해 알려줘”)
54
아래아 한글(.hwp 및 .hwpx) 파일을 잘 학습합니다.RAG 검색, LLM 검색, Web 검색으로 방대한 정보를 제공합니다.
72
너는 아직도 키워드로 검색하니? AI프로는 긴 문장으로 된 자연어 검색이 가능합니다.
47
검색 결과 페이지에서 핵심 내용을 요약하여 답변합니다.링크만 잔뜩 보여주는 옛날 검색 페이지는 이제 그만!
59
텍스트, 이미지, 파일, 동영상 등 멀티미디어 검색배너 이미지, 첨부파일의 본문 내용, 동영상 내용까지 모두 검색
63
구글 번역, 네이버 파파고보다 더 우수한 번역 품질(예: “나는

In [35]:
# 3단계: JSON-LD 데이터에서 추출
json_ld_scripts = soup.find_all('script', type='application/ld+json')
for script in json_ld_scripts:
    if script.string:
        data = json.loads(script.string)
        if isinstance(data, list):
            data = data[0] if data else {}
        
        text_fields = [
            data.get('description', ''),
            data.get('articleBody', ''),
            data.get('text', '')
        ]
        
        for text in text_fields:
            print(text.strip())

In [31]:
print(soup)

<!DOCTYPE html>

<html lang="ko">
<head>
<title>AI 프로 chatty.kr</title>
<link as="font" crossorigin="anonymous" href="https://fastly.jsdelivr.net/gh/Project-Noonnu/noonfonts_2107@1.1/Pretendard-Regular.woff" rel="preload" type="font/woff"/>
<meta charset="utf-8"/>
<link href="/css/animate.min.css" rel="stylesheet" type="text/css"/>
<link href="/css/swiper-bundle.min.css" rel="stylesheet" type="text/css"/>
<link href="/css/fonts.css" rel="stylesheet" type="text/css"/>
<link href="/css/layout2.css" rel="stylesheet" type="text/css"/>
<link href="/css/layout.css" rel="stylesheet" type="text/css"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/>
<meta content="맞춤형AI챗봇, Home, 학습형챗봇, 인공지능챗봇, 채팅 서비스, AI 챗봇 추천, 고객 맞춤형 AI, 채티, chatty" name="keywords"/>
<meta content="채티는 텍스트, 파일, 웹을 학습하여 나만을 위한 맞춤형 AI 대화를 제공합니다. 맞춤형 AI 챗봇 서비스로 고객 맞춤 대화를 경험해 보세요." name="description"/>
<meta content

In [6]:
import re
import chardet

In [13]:
async def _parse_page_content(soup: BeautifulSoup) -> str:
    """
    HTML에서 웹페이지 본문 내용을 추출하는 함수
    """
    try:
        # 1단계: 불필요한 태그들 제거
        # 스크립트, 스타일, 네비게이션, 광고 등 제거
        for element in soup(['script', 'style', 'nav', 'header', 'footer', 'aside', 
                           'advertisement', 'ads', 'sidebar', 'menu', 'breadcrumb']):
            element.decompose()
        
        # 클래스명/ID로 불필요한 요소들 제거
        unwanted_selectors = [
            '[class*="nav"]', '[class*="menu"]', '[class*="header"]', '[class*="footer"]',
            '[class*="sidebar"]', '[class*="aside"]', '[class*="widget"]', '[class*="ad"]',
            '[class*="advertisement"]', '[class*="banner"]', '[class*="popup"]',
            '[class*="cookie"]', '[class*="social"]', '[class*="share"]', '[class*="comment"]',
            '[id*="nav"]', '[id*="menu"]', '[id*="header"]', '[id*="footer"]',
            '[id*="sidebar"]', '[id*="aside"]', '[id*="widget"]', '[id*="ad"]'
        ]
        
        for selector in unwanted_selectors:
            for element in soup.select(selector):
                element.decompose()
        
        # 2단계: 주요 본문 컨테이너 찾기 (우선순위 순)
        main_content_selectors = [
            'main',
            'article',
            '[role="main"]',
            '.main-content',
            '.content',
            '.post-content',
            '.entry-content',
            '.article-content',
            '.page-content',
            '.body-content',
            '.main-body',
            '#content',
            '#main',
            '#main-content',
            '.container .content',
            '.wrapper .content'
        ]
        
        main_content = None
        for selector in main_content_selectors:
            element = soup.select_one(selector)
            if element:
                main_content = element
                print(f"🔍 본문 컨테이너 발견: {selector}")
                break
        
        # 3단계: 본문 컨테이너를 찾지 못한 경우 body 전체 사용
        if not main_content:
            main_content = soup.find('body')
            if not main_content:
                main_content = soup
        
        # 4단계: 텍스트 추출 및 정리
        if main_content:
            # 단락별로 텍스트 추출
            paragraphs = []
            
            # 제목들 추출 (h1-h6)
            for heading in main_content.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
                text = heading.get_text(strip=True)
                if text and len(text) > 10:  # 너무 짧은 제목 제외
                    paragraphs.append(text)
            
            # 단락들 추출 (p, div, section 등)
            for paragraph in main_content.find_all(['p', 'div', 'section', 'article']):
                text = paragraph.get_text(strip=True)
                if text and len(text) > 20:  # 너무 짧은 텍스트 제외
                    # 중복 제거 (이미 추가된 텍스트와 80% 이상 유사하면 제외)
                    is_duplicate = False
                    for existing in paragraphs:
                        if len(text) > 0 and len(existing) > 0:
                            # 간단한 유사도 체크 (공통 단어 비율)
                            text_words = set(text.split())
                            existing_words = set(existing.split())
                            if len(text_words) > 0:
                                similarity = len(text_words.intersection(existing_words)) / len(text_words)
                                if similarity > 0.8:
                                    is_duplicate = True
                                    break
                    
                    if not is_duplicate:
                        paragraphs.append(text)
            
            # 5단계: 본문 내용 조합 및 길이 제한
            if paragraphs:
                # 단락들을 합쳐서 본문 생성
                full_content = '\n\n'.join(paragraphs)
                
                # 길이 제한 (최대 2000자)
                if len(full_content) > 2000:
                    # 문장 단위로 자르기
                    sentences = full_content.split('.')
                    truncated_content = ""
                    for sentence in sentences:
                        if len(truncated_content + sentence + '.') <= 1950:
                            truncated_content += sentence + '.'
                        else:
                            break
                    
                    if truncated_content:
                        full_content = truncated_content + "..."
                    else:
                        full_content = full_content[:1950] + "..."
                
                # 연속된 공백 및 줄바꿈 정리
                full_content = re.sub(r'\s+', ' ', full_content)
                full_content = re.sub(r'\n\s*\n', '\n\n', full_content)
                
                print(f"🔍 본문 내용 추출 완료: {len(full_content)}자")
                return full_content.strip()
        
        # 6단계: 모든 방법이 실패한 경우 meta description 사용
        meta_desc = soup.select_one('meta[name="description"]')
        if meta_desc and meta_desc.get('content'):
            content = meta_desc.get('content').strip()
            if content:
                print(f"🔍 meta description 사용: {len(content)}자")
                return content
        
        # og:description 시도
        og_desc = soup.select_one('meta[property="og:description"]')
        if og_desc and og_desc.get('content'):
            content = og_desc.get('content').strip()
            if content:
                print(f"🔍 og:description 사용: {len(content)}자")
                return content
        
    except Exception as e:
        print(f"페이지 본문 파싱 중 오류: {str(e)}")
    
    return ""

In [7]:
async def _decode_html_content(html_bytes: bytes, content_type: str) -> str:
    """
    HTML 바이트를 적절한 인코딩으로 디코딩하는 함수
    """
    try:
        # 1단계: Content-Type 헤더에서 charset 확인
        encoding = None
        if content_type:
            import re
            charset_match = re.search(r'charset=([^;\s]+)', content_type.lower())
            if charset_match:
                encoding = charset_match.group(1).strip('"\'')
        
        # 2단계: HTML meta 태그에서 charset 확인 (첫 1024 바이트만)
        if not encoding:
            html_start = html_bytes[:1024].decode('ascii', errors='ignore').lower()
            # <meta charset="utf-8"> 형태
            charset_match = re.search(r'<meta[^>]+charset=["\']?([^"\'>\s]+)', html_start)
            if charset_match:
                encoding = charset_match.group(1)
            else:
                # <meta http-equiv="content-type" content="text/html; charset=euc-kr"> 형태
                content_match = re.search(r'<meta[^>]+http-equiv=["\']?content-type["\']?[^>]*content=["\'][^"\']*charset=([^"\'>\s;]+)', html_start)
                if content_match:
                    encoding = content_match.group(1)
        
        # 3단계: 인코딩이 확실하면 해당 인코딩으로 디코딩
        if encoding:
            try:
                # 일반적인 인코딩 별칭 처리
                encoding_aliases = {
                    'euc-kr': 'euc-kr',
                    'euckr': 'euc-kr', 
                    'ks_c_5601-1987': 'euc-kr',
                    'korean': 'euc-kr',
                    'cp949': 'cp949',
                    'ms949': 'cp949',
                    'utf-8': 'utf-8',
                    'utf8': 'utf-8'
                }
                encoding = encoding_aliases.get(encoding.lower(), encoding)
                return html_bytes.decode(encoding)
            except (UnicodeDecodeError, LookupError):
                print(f"지정된 인코딩 '{encoding}'으로 디코딩 실패, chardet 사용")
        
        # 4단계: chardet으로 인코딩 감지
        import chardet
        detected = chardet.detect(html_bytes)
        if detected and detected['encoding']:
            detected_encoding = detected['encoding']
            confidence = detected['confidence']
            print(f"chardet 감지 결과: {detected_encoding} (신뢰도: {confidence:.2f})")
            
            # 신뢰도가 0.7 이상이면 감지된 인코딩 사용
            if confidence >= 0.7:
                try:
                    return html_bytes.decode(detected_encoding)
                except UnicodeDecodeError:
                    print(f"chardet 감지 인코딩 '{detected_encoding}' 디코딩 실패")
        
        # 5단계: 한국 사이트의 일반적인 인코딩들 순서대로 시도
        fallback_encodings = ['euc-kr', 'cp949', 'utf-8', 'iso-8859-1']
        for encoding in fallback_encodings:
            try:
                return html_bytes.decode(encoding)
            except UnicodeDecodeError:
                continue
        
        # 최후 수단: errors='replace'로 UTF-8 디코딩
        print("모든 인코딩 시도 실패, UTF-8 강제 디코딩 (일부 문자 손실 가능)")
        return html_bytes.decode('utf-8', errors='replace')
        
    except Exception as e:
        print(f"HTML 디코딩 중 오류: {str(e)}")
        # 최후 수단
        return html_bytes.decode('utf-8', errors='replace')

In [14]:
# 페이지 본문 내용 파싱
page_content = await _parse_page_content(soup)

🔍 본문 내용 추출 완료: 1910자


In [15]:
print(page_content[0:500])

AI프로는 업무용 AI 솔루션입니다. 우리 기업∙기관에 딱 맞는 업무용 AI 솔루션! 약 100개의 기업과 기관에서 사용하고 있습니다. 당신의 업무에 AI 날개를 달아드립니다.우리 기업/기관을 위한 맞춤형 AI 솔루션, AI프로 맞춤형 AI 챗봇 사용 신청서 RAG 기술을 활용한 맞춤형 AI 챗봇 서비스 기본정보* 은 필수정보입니다. AI 채팅나보다 나를 더 잘 아는 RAG 기반의 똑똑한 AI 비서시나리오 기반의 기존 챗봇과 달리 답변 내용이 매우 우수합니다우리 기업∙기관의 데이터를 학습시킬 수 있습니다.(예: “우리 회사 김한국 과장에 대해 알려줘”)아래아 한글(.hwp 및 .hwpx) 파일을 잘 학습합니다.RAG 검색, LLM 검색, Web 검색으로 방대한 정보를 제공합니다.AI 검색너는 아직도 키워드로 검색하니?AI프로는 긴 문장으로 된 자연어 검색이 가능합니다.검색 결과 페이지에서 핵심 내용을 요약하여 답변합니다.링크만 잔뜩 보여주는 옛날 검색 페이지는 이제 그만!텍스트, 이


In [18]:
url3 = "https://www.joongang.co.kr/"
# 커스텀 헤더 설정 (네이버는 User-Agent 확인)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

async with aiohttp.ClientSession() as session:
    async with session.get(url3, headers=headers) as response:
        if response.status != 200:
            print(f"네이버 접근 실패: 상태 코드 {response.status}")
        
        html_content = await response.text()
        soup = BeautifulSoup(html_content, 'html.parser')

In [19]:
page_content = await _parse_page_content(soup)
print(page_content[0:500])


🔍 본문 컨테이너 발견: main
🔍 본문 내용 추출 완료: 1931자
조승현의 기쁨과 희망 세상을 바라보는 창대통령 이재명, 그의 삶과 정치'왜 졌을까'보다 중요한 것들李 "새 신도시 건설? 목 마르다고 소금물 계속 마시는 격"李 취임 한달 회견 "무너진 민생회복 전력" [모두발언 전문]李대통령 "검찰개혁 자업자득…기소에 맞춰 사건 조작 안돼"李 "노동시간 단축 반드시 필요, 주4.5일제 점진적으로 추진"李대통령 "감사원 기능, 지금이라도 국회 넘겨주고 싶다"민주 "李 대통령 회견, 정상 정부 들어섰음을 국민께 확인시켜"경제사회문화스포츠추천 Pick!중앙재테크박람회야구 비하인드이재명의 사람들강력계 25시호모 트레커스팩플hello! Parents노태우 비사오디오'뉴스페어링' 팟캐스트중앙재테크박람회호모 트레커스hello! Parents강력계 25시오는 8일쯤 한국을 찾을 예정이었던 마코 루비오 미 국무장관이 방한을 취소하면서 이를 한·미 정상회담 의제와 일정을 조율하는 기회로 삼으려던 정부의 계획에도 차질이 생겼다. 이재명 대통령이 아직 도널드 트럼프 미
